# Does using prior SQL queries help? 
In this workbook, we evalaute the effect of using "prior sql logs" in LLM prompts. For that, we will amend each table schema with the prior queries run on them. We need remove the query that is tested from the so called log. 


In [178]:
import pandas as pd
import tensorflow_hub as hub
import numpy as np
import altair as alt
from openai import OpenAI
from sql_metadata import Parser
import os
from IPython.display import display, Markdown
from collections import defaultdict 

In [179]:
df_q = pd.read_json('data/queries.json')

In [180]:
df_q

,question,sql,original_sql,tables_referred
0,How many organisation units in human resources...,SELECT COUNT(*) FROM (SELECT hr_org_unit_key F...,SELECT count(*) from (select hr_org_unit_key f...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]"
1,How many organisation units in human resources...,SELECT COUNT(*) FROM (SELECT hr_org_unit_key F...,SELECT count(*) from (select hr_org_unit_key f...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]"
2,Count the number of courses offered by each de...,"SELECT department_id, COUNT(*) AS course_count...","SELECT department_id, COUNT(*) AS course_count...",[CIS_COURSE_CATALOG]
3,"What are the past, upcoming, and current courses?","SELECT DISTINCT subject_title, effective_term_...","select distinct subject_title, effective_term_...","[CIS_COURSE_CATALOG, ACADEMIC_TERMS]"
4,Which faculty has had the most location changes?,SELECT MAX(cnt) FROM (SELECT COUNT(*) AS cnt F...,select max(cnt) from (select count(*) as cnt f...,[FCLT_BUILDING_ADDRESS_HIST]
5,What is the average number of courses a studen...,SELECT AVG(cnt) FROM (SELECT COUNT(*) AS cnt F...,select avg(cnt) from (select count(*) as cnt f...,"[MIT_STUDENT_DIRECTORY, SUBJECT_ENROLLABLE]"
6,Which students are employed by mit?,SELECT DISTINCT empl.full_name FROM mit_studen...,select distinct empl.full_name from mit_studen...,"[MIT_STUDENT_DIRECTORY, EMPLOYEE_DIRECTORY]"
7,What is the shortest course available this sem...,"SELECT DISTINCT subject_title, term_end_date -...","select distinct subject_title, term_end_date-t...","[ACADEMIC_TERMS, CIS_COURSE_CATALOG]"
8,Show buildings and their address.,"SELECT fb.building_name, fba.* FROM fac_buildi...","SELECT fb.building_name, fba.* FROM fac_buildi...","[FAC_BUILDING, FAC_BUILDING_ADDRESS]"
9,How many students do we have?,SELECT COUNT(*) FROM mit_student_directory;,select count(*) from mit_student_directory;,[MIT_STUDENT_DIRECTORY]


In [181]:
tables = []
for q in df_q['sql']: 
    t = Parser(q).tables
    tables.append([v.upper() for v in t])
df_q['tables'] = tables

In [182]:
unique_tables = set()
for  l in tables: 
    for t in l:
        unique_tables.add(t.upper())

In [183]:
len(unique_tables)

31

In [184]:
unique_tables

{'ACADEMIC_TERMS',
 'ACADEMIC_TERMS_ALL',
 'CIS_COURSE_CATALOG',
 'COURSE_CATALOG_SUBJECT_OFFERED',
 'EMPLOYEE_DIRECTORY',
 'FAC_BUILDING',
 'FAC_BUILDING_ADDRESS',
 'FCLT_BUILDING',
 'FCLT_BUILDING_ADDRESS_HIST',
 'FCLT_ROOMS',
 'HR_ORG_UNIT',
 'HR_ORG_UNIT_NEW',
 'IAP_SUBJECT_CATEGORY',
 'IAP_SUBJECT_DETAIL',
 'IAP_SUBJECT_PERSON',
 'IAP_SUBJECT_SESSION',
 'LIBRARY_MATERIAL_STATUS',
 'MIT_STUDENT_DIRECTORY',
 'SE_PERSON',
 'SPACE_UNIT',
 'STUDENT_DEPARTMENT',
 'SUBJECT_ENROLLABLE',
 'SUBJECT_OFFERED',
 'SUBJECT_SUMMARY',
 'TIP_DETAIL',
 'TIP_MATERIAL',
 'TIP_MATERIAL_STATUS',
 'TIP_SUBJECT_OFFERED',
 'TOP_LEVEL_DOMAIN',
 'ZIP_CANADA',
 'ZIP_USA'}

In [185]:
df_q

,question,sql,original_sql,tables_referred,tables
0,How many organisation units in human resources...,SELECT COUNT(*) FROM (SELECT hr_org_unit_key F...,SELECT count(*) from (select hr_org_unit_key f...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]","[HR_ORG_UNIT_NEW, HR_ORG_UNIT]"
1,How many organisation units in human resources...,SELECT COUNT(*) FROM (SELECT hr_org_unit_key F...,SELECT count(*) from (select hr_org_unit_key f...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]","[HR_ORG_UNIT_NEW, HR_ORG_UNIT]"
2,Count the number of courses offered by each de...,"SELECT department_id, COUNT(*) AS course_count...","SELECT department_id, COUNT(*) AS course_count...",[CIS_COURSE_CATALOG],[CIS_COURSE_CATALOG]
3,"What are the past, upcoming, and current courses?","SELECT DISTINCT subject_title, effective_term_...","select distinct subject_title, effective_term_...","[CIS_COURSE_CATALOG, ACADEMIC_TERMS]","[CIS_COURSE_CATALOG, ACADEMIC_TERMS]"
4,Which faculty has had the most location changes?,SELECT MAX(cnt) FROM (SELECT COUNT(*) AS cnt F...,select max(cnt) from (select count(*) as cnt f...,[FCLT_BUILDING_ADDRESS_HIST],[FCLT_BUILDING_ADDRESS_HIST]
5,What is the average number of courses a studen...,SELECT AVG(cnt) FROM (SELECT COUNT(*) AS cnt F...,select avg(cnt) from (select count(*) as cnt f...,"[MIT_STUDENT_DIRECTORY, SUBJECT_ENROLLABLE]","[MIT_STUDENT_DIRECTORY, SUBJECT_ENROLLABLE]"
6,Which students are employed by mit?,SELECT DISTINCT empl.full_name FROM mit_studen...,select distinct empl.full_name from mit_studen...,"[MIT_STUDENT_DIRECTORY, EMPLOYEE_DIRECTORY]","[MIT_STUDENT_DIRECTORY, EMPLOYEE_DIRECTORY]"
7,What is the shortest course available this sem...,"SELECT DISTINCT subject_title, term_end_date -...","select distinct subject_title, term_end_date-t...","[ACADEMIC_TERMS, CIS_COURSE_CATALOG]","[ACADEMIC_TERMS, CIS_COURSE_CATALOG]"
8,Show buildings and their address.,"SELECT fb.building_name, fba.* FROM fac_buildi...","SELECT fb.building_name, fba.* FROM fac_buildi...","[FAC_BUILDING, FAC_BUILDING_ADDRESS]","[FAC_BUILDING, FAC_BUILDING_ADDRESS]"
9,How many students do we have?,SELECT COUNT(*) FROM mit_student_directory;,select count(*) from mit_student_directory;,[MIT_STUDENT_DIRECTORY],[MIT_STUDENT_DIRECTORY]


In [186]:
df_x = pd.read_json('data/queries.json')

In [187]:
df_x['tables_referred'] 

0                        [HR_ORG_UNIT_NEW, HR_ORG_UNIT]
1                        [HR_ORG_UNIT_NEW, HR_ORG_UNIT]
2                                  [CIS_COURSE_CATALOG]
3                  [CIS_COURSE_CATALOG, ACADEMIC_TERMS]
4                          [FCLT_BUILDING_ADDRESS_HIST]
5           [MIT_STUDENT_DIRECTORY, SUBJECT_ENROLLABLE]
6           [MIT_STUDENT_DIRECTORY, EMPLOYEE_DIRECTORY]
7                  [ACADEMIC_TERMS, CIS_COURSE_CATALOG]
8                  [FAC_BUILDING, FAC_BUILDING_ADDRESS]
9                               [MIT_STUDENT_DIRECTORY]
10       [LIBRARY_MATERIAL_STATUS, TIP_MATERIAL_STATUS]
11                                [ZIP_USA, ZIP_CANADA]
12                                    [SUBJECT_OFFERED]
13                          [FCLT_ROOMS, FCLT_BUILDING]
14              [SPACE_UNIT, FCLT_ROOMS, FCLT_BUILDING]
15                                 [CIS_COURSE_CATALOG]
16                                 [CIS_COURSE_CATALOG]
17                                 [EMPLOYEE_DIR

In [188]:
len(unique_tables)

31

## Map tables to queries 

# Give ChatGPT the "database" schema as context and elicit its answer

## Build the schema prompt 

In [189]:
def table_to_sql_create(t, l=False): 
    schema_file='data/views/schema_as_table/' + t + '.csv'
    p = None
    if os.path.isfile(schema_file):
        df_t = pd.read_csv(schema_file)
        p = "CREATE TABLE " + t + "(\n"  
        p = p + '\n'.join(list(df_t[['COLUMN_NAME', 'DATA_TYPE']].apply(lambda row: ' '.join(row.astype(str)), axis=1)))
        p = p + ')\n'
    return p

In [193]:
def create_prompt(): 
    prompt = "*** FACTS ABOUT THE DATABASE ***\n\n" 
    prompt += "Fact#1: All MIT buildings are designated by numbers \n"
    prompt += "Fact#2: IAP stands for Independent Activities Period \n"
    prompt += "Fact#3: TIP stands for Textbook Information Provisioning \n"
    prompt += "Fact#4: CIS stands for Curricular Information System \n"
    prompt += "Fact#5: KRB stands for Kerberos \n\n"
    prompt += "*** ALL THE TABLES IN THE DATABASE *** \n\n"
    cnt = 0 
    for i, t in enumerate(unique_tables):
        p = table_to_sql_create(t, )
        if not p:
            print(t)
        else: 
            cnt = cnt + 1
            prompt = prompt + p
            prompt = prompt + "\n"
    return prompt 

In [194]:
print(create_prompt())

*** FACTS ABOUT THE DATABASE ***

Fact#1: All MIT buildings are designated by numbers 
Fact#2: IAP stands for Independent Activities Period 
Fact#3: TIP stands for Textbook Information Provisioning 
Fact#4: CIS stands for Curricular Information System 
Fact#5: KRB stands for Kerberos 

*** ALL THE TABLES IN THE DATABASE *** 

CREATE TABLE ACADEMIC_TERMS(
ACADEMIC_TERMS_KEY VARCHAR2
TERM_CODE VARCHAR2
TERM_DESCRIPTION VARCHAR2
TERM_SELECTOR VARCHAR2
TERM_START_DATE DATE
TERM_END_DATE DATE
ACADEMIC_YEAR VARCHAR2
ACADEMIC_YEAR_DESC VARCHAR2
IS_CURRENT_TERM VARCHAR2
IS_REGULAR_TERM VARCHAR2
TERM_STATUS_INDICATOR VARCHAR2
TERM_STATUS VARCHAR2
FINANCIAL_AID_YEAR VARCHAR2
DEGREE_YEAR VARCHAR2
LAST_DAY_OF_FINAL_EXAM DATE
PRE_REGISTRATION_START_DAY DATE
REGISTRATION_DAY DATE
FIRST_DAY_OF_CLASSES DATE
LAST_DAY_OF_CLASSES DATE
ADD_DATE DATE
DROP_DATE DATE
GRADUATE_AWARD_START_DATE DATE
GRADUATE_AWARD_END_DATE DATE
WAREHOUSE_LOAD_DATE DATE)

CREATE TABLE TIP_DETAIL(
TIP_SUBJECT_OFFERED_KEY VARCHAR

In [151]:
key = os.environ['OPENAI_API_KEY']

In [152]:
client = OpenAI(api_key=key)

In [153]:
gpt_sql_rule = []
for q in df_q['question']: 
    prompt = create_prompt()
    response = client.chat.completions.create(
      model="gpt-4",
      messages = [
          {
          "role": "system",
          # "content": "Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. \
          # The MIT Data Warehouse is a central data source that combines data from various administrative systems at MIT, containing \
          # information about students, faculty, and personnel. The schemas of all the tables that you might need \
          # for answering the user questions are below. There are 31 tables in the database.  Reply with only the answer \
          # in SQL and include no linebreaks, newlines, escape characters or other commentary.\n\n" + prompt    
          "content": "Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. \
          The MIT Data Warehouse is a central data source that combines data from various administrative systems at MIT, containing \
          information about students, faculty, and personnel. \
          Facts about the data in the database that you should take into account and the schemas of all the tables that you might need \
          for answering the user question are also given below. There are 31 tables in the database. \
          \
          Reply with only the answer in SQL and include no linebreaks, newlines, escape characters or other commentary.\n\n" + prompt  
        },
          {"role":"user", 
           "content":q}]
          )
    gpt_sql_rule.append(response.choices[0].message.content)

In [166]:
df_q['gpt_sql_rule'] = gpt_sql_rule

In [167]:
tables = []
for q in df_q['gpt_sql_rule']: 
    t = Parser(q).tables
    tables.append([v.upper() for v in t])
df_q['gpt_sql_rule_tables'] = tables


In [168]:
accuracy = [] 
for i, t in enumerate(df_q['tables']):
    accuracy.append(len(set(t).intersection(df_q['gpt_sql_rule_tables'][i]))/len(t))

In [169]:
sum(np.array(accuracy)==1.0)

14

In [170]:
list(df_q['sql'][np.array(accuracy)==0.5])

['SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit_new INTERSECT SELECT hr_org_unit_key FROM hr_org_unit);',
 'SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit_new MINUS SELECT hr_org_unit_key FROM hr_org_unit);',
 "SELECT DISTINCT subject_title, effective_term_code, CASE WHEN term_start_date > SYSDATE THEN 'Upcoming' WHEN term_start_date <= SYSDATE AND term_end_date > SYSDATE THEN 'Current' ELSE 'Past' END AS term_status FROM cis_course_catalog, academic_terms WHERE effective_term_code = term_code;",
 'SELECT DISTINCT empl.full_name FROM mit_student_directory mit, employee_directory empl WHERE empl.full_name = mit.full_name;',
 'SELECT DISTINCT subject_title, term_end_date - term_start_date FROM (SELECT MIN(term_end_date - term_start_date) AS min_duration FROM academic_terms), academic_terms, cis_course_catalog WHERE effective_term_code = term_code AND term_end_date - term_start_date = min_duration;',
 'SELECT DISTINCT library_material_status FROM library_mat

In [171]:
list(df_q['question'][np.array(accuracy)==0.5])

['How many organisation units in human resources are relabeled?',
 'How many organisation units in human resources are new and not relabeled?',
 'What are the past, upcoming, and current courses?',
 'Which students are employed by mit?',
 'What is the shortest course available this semester?',
 'Show all statuses for materials?',
 'Count the number of room in a building.',
 'Show the name for subjects which are enrollable for Computer Science Student.',
 'Find the city with the post code 501.',
 'Which department has the most kinds of payroll ranks?']

In [172]:
list(df_q['tables'][np.array(accuracy)==0.5])

[['HR_ORG_UNIT_NEW', 'HR_ORG_UNIT'],
 ['HR_ORG_UNIT_NEW', 'HR_ORG_UNIT'],
 ['CIS_COURSE_CATALOG', 'ACADEMIC_TERMS'],
 ['MIT_STUDENT_DIRECTORY', 'EMPLOYEE_DIRECTORY'],
 ['ACADEMIC_TERMS', 'CIS_COURSE_CATALOG'],
 ['LIBRARY_MATERIAL_STATUS', 'TIP_MATERIAL_STATUS'],
 ['FCLT_ROOMS', 'FCLT_BUILDING'],
 ['SUBJECT_SUMMARY', 'SUBJECT_ENROLLABLE'],
 ['ZIP_USA', 'ZIP_CANADA'],
 ['SE_PERSON', 'EMPLOYEE_DIRECTORY']]

In [173]:
list(df_q['question'][np.array(accuracy)==0.5])

['How many organisation units in human resources are relabeled?',
 'How many organisation units in human resources are new and not relabeled?',
 'What are the past, upcoming, and current courses?',
 'Which students are employed by mit?',
 'What is the shortest course available this semester?',
 'Show all statuses for materials?',
 'Count the number of room in a building.',
 'Show the name for subjects which are enrollable for Computer Science Student.',
 'Find the city with the post code 501.',
 'Which department has the most kinds of payroll ranks?']

In [174]:
list(df_q['sql'][np.array(accuracy)==0.5])

['SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit_new INTERSECT SELECT hr_org_unit_key FROM hr_org_unit);',
 'SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit_new MINUS SELECT hr_org_unit_key FROM hr_org_unit);',
 "SELECT DISTINCT subject_title, effective_term_code, CASE WHEN term_start_date > SYSDATE THEN 'Upcoming' WHEN term_start_date <= SYSDATE AND term_end_date > SYSDATE THEN 'Current' ELSE 'Past' END AS term_status FROM cis_course_catalog, academic_terms WHERE effective_term_code = term_code;",
 'SELECT DISTINCT empl.full_name FROM mit_student_directory mit, employee_directory empl WHERE empl.full_name = mit.full_name;',
 'SELECT DISTINCT subject_title, term_end_date - term_start_date FROM (SELECT MIN(term_end_date - term_start_date) AS min_duration FROM academic_terms), academic_terms, cis_course_catalog WHERE effective_term_code = term_code AND term_end_date - term_start_date = min_duration;',
 'SELECT DISTINCT library_material_status FROM library_mat

In [175]:
list(df_q['gpt_sql_rule'][np.array(accuracy)==0.5])

['SELECT COUNT(*) FROM HR_ORG_UNIT WHERE HR_ORG_UNIT_TITLE != HR_DEPARTMENT_NAME;',
 "SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_ORG_UNIT_TITLE NOT LIKE '%relabel%'",
 "SELECT TERM_CODE, SUBJECT_ID, SUBJECT_TITLE FROM SUBJECT_OFFERED WHERE TERM_CODE IN (SELECT TERM_CODE FROM ACADEMIC_TERMS WHERE IS_CURRENT_TERM = 'Y' OR TERM_START_DATE > CURRENT_DATE OR TERM_END_DATE < CURRENT_DATE)",
 'SELECT MIT_STUDENT_DIRECTORY.FULL_NAME FROM MIT_STUDENT_DIRECTORY INNER JOIN SE_PERSON ON MIT_STUDENT_DIRECTORY.FULL_NAME = SE_PERSON.FULL_NAME WHERE SE_PERSON.EMPLOYEE_TYPE IS NOT NULL;',
 "SELECT SUBJECT_ID, SUBJECT_TITLE, MIN(TERM_DURATION) AS SHORTEST_DURATION FROM COURSE_CATALOG_SUBJECT_OFFERED WHERE IS_OFFERED_THIS_YEAR='Y' AND TERM_CODE=(SELECT TERM_CODE FROM ACADEMIC_TERMS WHERE IS_CURRENT_TERM='Y')",
 'SELECT TIP_MATERIAL_STATUS FROM TIP_MATERIAL_STATUS;',
 "SELECT COUNT(*) FROM FCLT_ROOMS WHERE FCLT_BUILDING_KEY = 'YourBuildingKey'",
 "SELECT SUBJECT_TITLE FROM SUBJECT_ENROLLABLE WHERE OFFE

In [176]:
np.mean(np.array(accuracy))

0.5472972972972973

In [177]:
tp = 0 
fp = 0 
fn = 0 
accurate = 0 
for i, t in enumerate(df_q['tables']):
    tp = tp + len(set(t).intersection(tables[i]))
    fn = fn + len(set(t).difference(tables[i]))
    fp = fp + len(set(tables[i]).difference(t))
    accurate = accurate + (1 if set(t) == set(tables[i]) else 0) 
precision = tp / (tp + fp) 
recall = tp / (tp + fn)
accuracy = accurate / len(tables) 
print("precision:{0}".format(precision))
print("recall:{0}".format(recall))
print("f1:{0}".format(2*precision*recall/(precision + recall)))
print("accuracy:{0}".format(accuracy))

precision:0.625
recall:0.5084745762711864
f1:0.5607476635514018
accuracy:0.35135135135135137
